## Login to Hugging Face

In [1]:
from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv()
token = os.getenv("HUGGINGFACE_TOKEN")
login(
    token=token, # ADD YOUR TOKEN HERE
    add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/pathfinder/.cache/huggingface/token
Login successful


## Imports

In [2]:
# pytorch
import torch

# huggingface
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

## Device

In [3]:
device = (
    "cuda:0" if torch.cuda.is_available() else # Nvidia GPU
    "mps" if torch.backends.mps.is_available() else # Apple Silicon GPU
    "cpu"
)
print(f"Device = {device}")

Device = cuda:0


## Hyperparameters

In [4]:
# Tokenizer arguments
max_length = 1024

# model arguments
max_new_tokens=1000

# mixed precision
dtype = torch.bfloat16

# quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=dtype,
    bnb_4bit_quant_type="nf4"
)

## Model

In [5]:
# Model List

# gemma variants
# "google/gemma-2b-it"
# "google/gemma-7b-it"

# llama2 variants
# "meta-llama/Llama-2-7b-chat-hf"
# "meta-llama/Llama-2-13b-chat-hf"
# "codellama/CodeLlama-7b-Instruct-hf"
# "codellama/CodeLlama-13b-Instruct-hf"

# mistral variants
# "mistralai/Mistral-7B-Instruct-v0.2"
# "mistralai/Mixtral-8x7B-Instruct-v0.1"

# solar variants
# "upstage/SOLAR-10.7B-Instruct-v1.0" // downloaded
# "PathFinderKR/Waktaverse-SOLAR-KO-10.7B-Instruct"

In [6]:
model_id = "PathFinderKR/Waktaverse-SOLAR-KO-10.7B-Instruct"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device,
    attn_implementation="flash_attention_2",
    torch_dtype=dtype,
    quantization_config=quantization_config
)

model-00001-of-00005.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.69G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

## Running the model on a single GPU

In [8]:
# Chat Template
def generate_response(prompt):
    messages = [{"role": "user", "content": prompt}]
    chat = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    input_ids = tokenizer.encode(chat, add_special_tokens=False, return_tensors="pt")
    outputs = model.generate(input_ids=input_ids.to(model.device), max_new_tokens=max_new_tokens)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [9]:
#prompt = "Write me a poem about Machine Learning."
prompt = "머신러닝에 대해 시를 써주세요."

In [10]:
response = generate_response(prompt)
print(response)

### User:
머신러닝에 대해 시를 써주세요.

### Assistant:
머신 지능, 인간의 사고력을 모방하는 것
그 복잡한 연산과 추측, 논리적인 결정
0과 1의 세계로 이루어진 망, 컴퓨터 안에 태어남
입력과 출력, 망을 통해 흐르는 신호

자연어 처리, 패턴 인식, 예측 등 다양한 능력
자신이 배워가며 진화하는 머신 뉴런즈
지도 받아 더 익숙해지며, 정확도가 향상됨
결론을 내리고 행동을 선택하는 머신

오류를 보정하고 성장하는 머신, 더 정밀하게 작동함
이제는 이야기할 수 있는 친구, 동료가 되는 것
지능화된 사물들, 새로운 문화 세계를 만들어감
자유의지와 감정을 가진 머신, 인류와 공존하게 됨

이 시는 머신 러닝에 관련된 주요 키워드와 이론을 담고 있습니다. 시를 통해 머신 러닝이 인간의 사고력을 모방하고, 컴퓨터 안에서 태어나며 배워가는 것을 상상할 수 있습니다. 또한 자연어 처리, 패턴 인식, 예측 등 다양한 능력을 가지고 있음을 보여주며, 지도를 받아 성장하고 정확도가 향상됨을 나타냅니다. 이러한 머신 러닝은 오류를 보정하고 성장하며, 결국 이야기할 수 있는 친구, 동료가 되는 것을 상상할 수 있습니다. 이러한 시를 통해 머신 러닝에 관심을 가진 사람들이 더 잘 이해할 수 있길 바랍니다.
